In [ ]:
import pandas as pd
import random
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

In [ ]:
dataset = load_dataset("argilla/reward-model-data-falcon")


def create_dataset(row):
  if row['choose-best']['value'][0]==2:
    row['response-1'],row['response-2']=row['response-2'],row['response-1']
  return row

dataset=dataset.map(lambda x: create_dataset(x))

In [ ]:
def loss_function(preferred_response_reward, alternate_response_reward):
    return -torch.mean(torch.log(torch.sigmoid(alternate_response_rewar

In [ ]:
from transformers import GPT2Model, GPT2Tokenizer

class GPT2RewardModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = GPT2Model.from_pretrained('gpt2')
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.regression_head = torch.nn.Linear(768, 1)

    def forward(self, context, response):

        entire_text = context + response
        context_dict = self.tokenizer(
            '<|startoftext|>' + entire_text + '<|endoftext|>'
        )

        input_ids = torch.tensor(context_dict.input_ids)
        attention_mask = torch.tensor(context_dict.attention_mask)


        gpt2_outputs = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        all_output_vectors = gpt2_outputs.last_hidden_state
        last_output_vector = all_output_vectors[-1]


        last_output_vector = last_output_vector.unsqueeze(0)
        reward = self.regression_head(last_output_vector)

        return reward
model = GPT2RewardModel()

**Training**

In [ ]:
def train(epochs=10):

    # Create the optimizer
    optimizer = torch.optim.Adam(
        model.parameters(), lr=1e-5)


    batch_idx = 0
    # Train the model
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}")
        for batch in tqdm(dataset):

            # Get the data
            prompt, preferred_reponse, alternate_response,choose,external_id = batch
            preferred_response_reward = model(prompt, preferred_reponse)
            alternate_response_reward = model(prompt, alternate_response)

            loss = loss_function(preferred_response_reward, alternate_response_reward)

            # Backward pass
            loss.backward()
            optimizer.step()

            # Zero the gradients
            optimizer.zero_grad()

            # Log the loss
            print(f"Loss: {loss.item()}", batch_idx)
            batch_idx += 1


train()